# Best score solution: восстановление пробелов

Идея решения:
- Сначала ставим очевидные разрывы по правилам: пунктуация (после запятой и т.п.), длинное тире с двух сторон, смена алфавита (кириллица/латиница), буква↔цифра, переход `a→A` (эффект CamelCase), предлоги/союзы «в/к/на/по/из/до/от/за/во/со/об» перед ПРОПИСНОЙ или числом, стартовые якоря домена (куплю/ищу/сдам/продам/новый/срочно…).
- Для непрерывных кириллических фрагментов применяем динамическую сегментацию (DP): минимизируем сумму стоимостей слов. Стоимость токена: `10 - Zipf(token)` из `wordfreq`; для очень длинных и редких токенов добавляется штраф.
- На выходе получаем отсортированный список позиций.

Ввод: `data/dataset.txt` (колонки `id,text_no_spaces`). 

Вывод: `submission.csv`. 


In [1]:
# Импорты и конфигурация
import os
from typing import List, Set, Tuple
from tqdm.auto import tqdm
from wordfreq import zipf_frequency

# Пути к файлам
INPUT_PATH = os.path.join('data', 'dataset.txt')
OUTPUT_PATH = 'submission.csv'

# Максимальная длина русского "слова" в окне DP
MAX_RU_WORD_LEN = 24


/Users/dmitriy/programming/avito-test-task/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Вспомогательные константы

DASHES = "—–"
PUNCT_THAT_NEEDS_SPACE_AFTER = set(",;!?)]}»…")
COMMON_START_WORDS = [
    "куплю","ищу","сдам","сдаю","продам","продаю","отдам",
    "новый","новая","новое","новые","срочно","хочу","нужен","нужно","нужна","срочнопродам",
]
ONE_LETTER_PREPS = set(list("вксоуюоВОСУЮО"))
TWO_LETTER_PREPS = ["на","по","до","от","из","во","со","об","за"]

In [3]:
# Вспомогательные функции для классов символов и правил

def is_cyr(ch: str) -> bool:
    return ("А" <= ch <= "Я") or ("а" <= ch <= "я") or ch in "Ёё"


def is_lat(ch: str) -> bool:
    return ("A" <= ch <= "Z") or ("a" <= ch <= "z")


def is_upper(ch: str) -> bool:
    return ch.upper() == ch and ch.lower() != ch


def is_lower(ch: str) -> bool:
    return ch.lower() == ch and ch.upper() != ch


def is_letter(ch: str) -> bool:
    return is_cyr(ch) or is_lat(ch)
    


In [4]:
# should_break — решает, нужно ли ставить пробел перед символом s[i]
#    по правилам пунктуации, тире, смены типов символов (буква/цифра,
#    кириллица/латиница) и перехода нижний→верхний регистр.
# add_initial_anchors — добавляет «якорные» разрывы после типовых
#    стартовых слов домена объявлений и по эвристике окончания «у/ю».
# add_preposition_gaps — расставляет пробелы вокруг коротких предлогов
#    («в/к/на/по/…») при последующей ПРОПИСНОЙ букве или числе.

def should_break(s: str, i: int) -> bool:
    prev = s[i - 1]
    cur = s[i]
    if prev in PUNCT_THAT_NEEDS_SPACE_AFTER:
        return True
    if prev == ".":
        if i >= 2 and s[i-2].isdigit() and cur.isdigit():
            return False
        if cur == ".":
            return False
        return True
    if cur in DASHES:
        if is_letter(prev) or prev.isdigit() or prev in ")]}'»":
            return True
    if prev in DASHES:
        if is_letter(cur) or cur.isdigit() or cur in "([«":
            return True
    if (is_letter(prev) and cur.isdigit()) or (prev.isdigit() and is_letter(cur)):
        return True
    if (is_cyr(prev) and is_lat(cur)) or (is_lat(prev) and is_cyr(cur)):
        return True
    if is_lower(prev) and is_upper(cur):
        return True
    return False


def add_initial_anchors(s: str, positions: Set[int]) -> None:
    for w in COMMON_START_WORDS:
        if s.startswith(w) and len(s) > len(w):
            positions.add(len(w))
    for k in range(2, min(10, len(s))):
        if s[k-1] in "ую" and is_cyr(s[k]):
            positions.add(k)
            break


def add_preposition_gaps(s: str, positions: Set[int]) -> None:
    n = len(s)
    for i in range(1, n-1):
        ch = s[i]
        if ch in ONE_LETTER_PREPS:
            nxt = s[i+1]
            if is_upper(nxt) or nxt.isdigit():
                if is_letter(s[i-1]) or s[i-1].isdigit():
                    positions.add(i); positions.add(i+1)
    for i in range(2, n-1):
        pair = s[i-2:i]
        if pair.lower() in TWO_LETTER_PREPS:
            nxt = s[i]
            if is_upper(nxt) or nxt.isdigit():
                if is_letter(s[i-3]) or s[i-3].isdigit():
                    positions.add(i-2); positions.add(i)



In [5]:
# ru_token_cost: вычисляет «цену» токена. Чем чаще слово в корпусах (выше Zipf),
#   тем ниже цена: cost = 10 - zipf. Также добавляется штраф за очень длинные редкие токены.
# segment_cyr_run_dp: динамическое программирование по подстроке из кириллицы.
#   Перебираем окончания слова с окном до MAX_RU_WORD_LEN, находим разбиение с минимальной суммой цен.
#   Возвращаем локальные индексы разрывов внутри run (без 0 и len(run)).


def ru_token_cost(token: str) -> float:
    t = token.lower()
    try:
        zipf = zipf_frequency(t, "ru")
    except Exception:
        zipf = 0.0

    if len(t) == 1 and t in {"и","в","с","к","у","я","а","о"}:
        zipf = max(zipf, 6.0)
    if len(t) == 2 and t in {"на","по","до","от","из","за","во","со","об"}:
        zipf = max(zipf, 5.2)

    cost = 10.0 - zipf
    if zipf < 1.0 and len(t) >= 8:
        cost += 2.0
    return cost


def segment_cyr_run_dp(run: str) -> List[int]:
    n = len(run)
    if n <= 1:
        return []
    dp: List[Tuple[float, int]] = [(float("inf"), -1) for _ in range(n + 1)]
    dp[0] = (0.0, -1)
    for i in range(1, n + 1):
        best_cost = float("inf"); best_j = -1
        max_k = min(MAX_RU_WORD_LEN, i)
        for k in range(1, max_k + 1):
            j = i - k
            token = run[j:i]
            cost = dp[j][0] + ru_token_cost(token)
            if cost < best_cost:
                best_cost, best_j = cost, j
        dp[i] = (best_cost, best_j)

    splits: List[int] = []
    i = n
    while i > 0:
        j = dp[i][1]
        if j <= 0:
            i = j
        else:
            splits.append(j); i = j
    return sorted(set(x for x in splits if 0 < x < n))


In [6]:
# add_dp_segmentation_for_cyr: выделяет кириллические подрядки и применяет к ним DP,
#   добавляя локальные разрывы в глобальный набор.
# predict_positions_for_text: объединяет правила (пунктуация/тире/тип символа),
#   стартовые якоря, короткие предлоги и DP. Возвращает отсортированный список позиций.

def add_dp_segmentation_for_cyr(s: str, positions: Set[int]) -> None:
    i = 0; n = len(s)
    while i < n:
        if is_cyr(s[i]):
            st = i
            while i < n and is_cyr(s[i]):
                i += 1
            run = s[st:i]
            if len(run) >= 4:
                for r in segment_cyr_run_dp(run):
                    positions.add(st + r)
        else:
            i += 1


def predict_positions_for_text(s: str) -> List[int]:
    if not s:
        return []
    positions: Set[int] = set()
    for i in range(1, len(s)):
        if should_break(s, i):
            positions.add(i)
    add_initial_anchors(s, positions)
    add_preposition_gaps(s, positions)
    add_dp_segmentation_for_cyr(s, positions)
    return sorted(i for i in positions if 0 < i < len(s))


In [7]:
# Функция для чтения датасета (pd.read_csv не работает, поскльку в данных есть строки с запятыми)

def read_dataset(path: str) -> List[Tuple[int, str]]:
    rows: List[Tuple[int, str]] = []
    with open(path, 'r', encoding='utf-8') as f:
        _ = f.readline()
        for line in f:
            if not line.strip():
                continue
            idx, text = line.rstrip('\n').split(',', 1)
            rows.append((int(idx), text))
    return rows


In [8]:
# Инференс и сохранение submission

rows = read_dataset(INPUT_PATH)

out_lines = ["id,predicted_positions"]
for row_id, text in tqdm(rows):
    pos = predict_positions_for_text(text)
    out_lines.append(f'{row_id},"{str(pos)}"')

with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    f.write("\n".join(out_lines) + "\n")

print(f"Saved {OUTPUT_PATH} (rows={len(rows)})")


100%|██████████| 1005/1005 [00:12<00:00, 79.50it/s]

Saved submission.csv (rows=1005)
